### PFSS Modeling Calculations

Tamar Ervin - September 26, 2022

Based off example notebook: Sam Badman - 8/25/22

This notebook goes through running a potential field source surface (PFSS, [1](https://ui.adsabs.harvard.edu/abs/1969SoPh....9..131A/abstract),[2](https://ui.adsabs.harvard.edu/abs/1969SoPh....6..442S/abstract)) model of the solar coronal magnetic field using pfsspy and accessing its components including tracing fieldlines, creating a map of coronal holes and flying a model spacecraft through it and tracing field line connections

Read more about pfsspy here : https://pfsspy.readthedocs.io/en/stable/ and view the source code here : https://github.com/dstansby/pfsspy/tree/0c5b78a2901c5dd55ddc286ba25a91717e85a40c

In [1]:
import astropy
import sunpy, sunpy.map
import pfsspy
import matplotlib.pyplot as plt
import numpy as np

import sys, os

sys.path.append(os.path.realpath(''))
# sys.path.append('/Users/tamarervin/Desktop/Repositories/mag_model/')
import utilities as utils
import psp_funcs as psp_funcs
import pfss_funcs as pfss_funcs
import pandas as pd

In [2]:
rss_values = np.arange(2.0, 3.75, 0.25)
IMG_DIR = os.path.realpath('../mag_model/images')
PSP_DATA_DIR = os.path.realpath('../mag_model/data/psp')
HCS_DATA_DIR = os.path.realpath('../mag_model/data/gong')


# Looking at PSP Data
- read in the PSP text file with timestamps and $\mathrm{B_r \, AU^2}$ values
- use to compare with data to find a 'best' magnetogram
- pull the polarity to overlay on the plot
- save lat, lon, and polarity values

In [3]:
# read in the PSP text file and convert to a pandas dataframe
import pandas as pd

psp_txt_file = '/Users/tamarervin/Desktop/Repositories/mag_model/data/psp/BrAU2_mode.csv'
psp_data = pd.read_csv(psp_txt_file, names=['datetime', 'Br'], engine='python')

# split the datetime column into a separate date and time column
psp_data[['date', 'time']] = psp_data.datetime.str.split('/', expand=True)
dates = psp_data.date + 'T' + psp_data.time

# convert datetime to a JD date and datetime object -- use the conversion function in utilities.py
dates_list = [utils.get_dates(date) for date in dates]
jd_dates = [d[2] for d in dates_list]
jd_dates = [d.to_value('jd', 'long') for d in jd_dates]
psp_data['jd'] = jd_dates
psp_data['date_obj'] = [d[1] for d in dates_list]

# add a polarity column
psp_data['polarity'] = np.sign(psp_data.Br)

# remove the data from september 5 due to the solar flare -- set up a flag column
date_str, date_obj, date_jd = utils.get_dates('2022-09-05T00:00:00.000')
flag_inds = np.logical_and(psp_data.date >= date_jd - 0.2, psp_data.date <= date_jd + 1)
flag = np.array(['' for x in range(len(psp_data.datetime))])
flag[flag_inds] = 'flare'
psp_data['flag'] = flag



# Find the trajectory of PSP using astrospice and timestamps above

In [91]:
# get the corresponding coordinates to the timestamps in the dataframe
# psp_coords_carr = psp_funcs.get_psp_coords(psp_data.date_obj)
import astrospice

kernels = astrospice.registry.get_kernels('psp', 'predict')
psp_coords_inertial = astrospice.generate_coords('SOLAR PROBE PLUS', psp_data.date_obj)

# Transform to Heliographic Carrington, i.e. the frame that co-rotates with the Sun.
psp_coords_carr = psp_coords_inertial.transform_to(
sunpy.coordinates.HeliographicCarrington(observer="self"))


Files Downloaded:   0%|          | 0/1 [00:00<?, ?file/s]

In [116]:
# do the parker spiral projection to get the coordinates at the source surface for different source surface heights
from astropy.table import QTable
# create QTable with astropy
# t = QTable()
coords = pd.DataFrame()
for rss in rss_values:
    # get psp at source surface
    psp_at_source_surface = psp_funcs.coord_projection(psp_coords_carr, rss, psp_data.date_obj)

    # save these values with corresponding source surface height as column name to the data array
    coords['lon_' + str(int(rss*100))] = psp_at_source_surface.lon.degree
    coords['lat_' + str(int(rss*100))] = psp_at_source_surface.lat.degree


# savecsv = os.path.join(PSP_DATA_DIR, 'psp_ss.csv')
# t.to_csv(savecsv)  

# create $\mathrm{B_rR^2}$ map
- create the PFSS model for one magnetogram at perihelion 
- run the PSP trajectory over the PFSS model and trace the field lines back down to the source surface using parker spiral
- get the Br values at the source surface from the PFSS model

In [93]:
# get a magnetogram for the specific date and time in question
filenames = ['adapt41311_03k012_202209170000_i00005600n1.fts.gz', 
'adapt41311_03k012_202209180000_i00005700n1.fts.gz', 'adapt40311_03k012_202209190000_i00005600n1.fts.gz', 
'adapt41311_03k012_202209200000_i00005600n1.fts.gz', 'adapt41311_03k012_202209210000_i00005600n1.fts.gz']	

#'adapt40311_03k012_202209190200_i00025600n1.fts.gz', 'adapt40311_03k012_202209190400_i00010500n1.fts.gz',
# 'adapt40311_03k012_202209190600_i00030500n1.fts.gz', 'adapt40311_03k012_202209190800_i00003600n1.fts.gz', 'adapt40311_03k012_202209191000_i00023600n1.fts.gz', 
# 'adapt40311_03k012_202209191200_i00043600n1.fts.gz', 'adapt40311_03k012_202209191400_i00005600n1.fts.gz', 'adapt40311_03k012_202209191600_i00005600n1.fts.gz', 


# create dataframe
hcs_data = []

In [101]:
for filename in filenames:

    # download magnetogram
    remote_path = "ftp://gong.nso.edu//adapt/maps/gong/2022"
    local_path = "data/gong"
    if not os.path.exists(f"{local_path}/{filename}") : # check if file has already been downloaded
        print(f"Downloading {local_path}/{filename}") 
        os.system(f'wget {remote_path}/{filename} -P {local_path}')
    else : print(f"{local_path}/{filename} already exists!" )

    # get an adapt magnetogram
    filepath = f"{local_path}/{filename}"
    adapt_magnetogram = pfss_funcs.adapt2pfsspy(filepath, return_magnetogram=True)

    # run pfss model for varying RSS values
    for rss in rss_values:
        # run the PFSS model with rss 
        pfss_model = pfss_funcs.adapt2pfsspy(filepath, rss)

        # magnetic field line tracing starting from photosphere
        flines = pfss_funcs.pfss2flines(pfss_model)

        # get Br at the source surface from the pfss model
        pfss_br = pfss_model.source_surface_br

        # get HCS
        hcs = [pfss_model.source_surface_pils[0].lon, pfss_model.source_surface_pils[0].lat]

        # add this to dataframe
        df_name = str(int(rss*100)) + "_" + str(pfss_br.date)
        hcs_df = pd.DataFrame({str(df_name) : hcs})
        hcs_data.append(hcs_df)

# concatenate dataframes
hcs_csv = pd.concat(hcs_data, axis=1)

data/gong/adapt41311_03k012_202209170000_i00005600n1.fts.gz already exists!
INFO: Missing metadata for solar radius: assuming the standard radius of the photosphere. [sunpy.map.mapbase]


For frame 'heliographic_stonyhurst' the following metadata is missing: dsun_obs,hgln_obs,hglt_obs
For frame 'heliographic_carrington' the following metadata is missing: dsun_obs,crlt_obs,crln_obs
 [sunpy.map.mapbase]


INFO: Missing metadata for solar radius: assuming the standard radius of the photosphere. [sunpy.map.mapbase]


For frame 'heliographic_stonyhurst' the following metadata is missing: dsun_obs,hgln_obs,hglt_obs
For frame 'heliographic_carrington' the following metadata is missing: dsun_obs,crlt_obs,crln_obs
 [sunpy.map.mapbase]
/opt/anaconda3/envs/mag_model/lib/python3.8/site-packages/pfsspy/tracing.py:180: UserWarning: At least one field line ran out of steps during tracing.
You should probably increase max_steps (currently set to 1000) and try again.
  warnings.warn(


INFO: Missing metadata for solar radius: assuming the standard radius of the photosphere. [sunpy.map.mapbase]


...

For frame 'heliographic_stonyhurst' the following metadata is missing: dsun_obs,hgln_obs,hglt_obs
For frame 'heliographic_carrington' the following metadata is missing: dsun_obs,crlt_obs,crln_obs
 [sunpy.map.mapbase]
/opt/anaconda3/envs/mag_model/lib/python3.8/site-packages/pfsspy/tracing.py:180: UserWarning: At least one field line ran out of steps during tracing.
You should probably increase max_steps (currently set to 1000) and try again.
  warnings.warn(


INFO: Missing metadata for solar radius: assuming the standard radius of the photosphere. [sunpy.map.mapbase]


For frame 'heliographic_stonyhurst' the following metadata is missing: dsun_obs,hgln_obs,hglt_obs
For frame 'heliographic_carrington' the following metadata is missing: dsun_obs,crlt_obs,crln_obs
 [sunpy.map.mapbase]
/opt/anaconda3/envs/mag_model/lib/python3.8/site-packages/pfsspy/tracing.py:180: UserWarning: At least one field line ran out of steps during tracing.
You should probably increase max_steps (currently set to 1000) and try again.
  warnings.warn(


INFO: Missing metadata for solar radius: assuming the standard radius of the photosphere. [sunpy.map.mapbase]


For frame 'heliographic_stonyhurst' the following metadata is missing: dsun_obs,hgln_obs,hglt_obs
For frame 'heliographic_carrington' the following metadata is missing: dsun_obs,crlt_obs,crln_obs
 [sunpy.map.mapbase]
/opt/anaconda3/envs/mag_model/lib/python3.8/site-packages/pfsspy/tracing.py:180: UserWarning: At least one field line ran out of steps during tracing.
You should probably increase max_steps (currently set to 1000) and try again.
  warnings.warn(


In [87]:
# save dataframe to CSV
# save this data array to a csv file
savecsv = os.path.join(HCS_DATA_DIR, 'hcs_data.csv')
hcs_csv.to_csv(savecsv, index=False)

NameError: name 'hcs_csv' is not defined

In [121]:
hcs_data[0]

,200.0_2022-09-17T00:00:00.000
0,"[143d30m00s, 144d05m03.65757087s, 144d30m00s, ..."
1,"[-24d32m23.87023331s, -23d55m33.40485755s, -23..."


In [ ]:
import astropy.units as u

# get Br at the source surface from the pfss model
pfss_br = pfss_model.source_surface_br

# get HCS
hcs = pfss_model.source_surface_pils[0]

# plot this using sunpy
fig = plt.figure(figsize=(10,5))
ax = fig.add_subplot()
# pfss_br.plot(axes=ax)

# plot the HCS
ax.scatter(hcs.lon, hcs.lat, color='black', s=3, label='HCS')

# plot the PSP trajectory on top of it
pos = np.where(psp_data.polarity == 1)
neg = np.where(psp_data.polarity == -1)

ax.scatter(psp_at_source_surface.lon[neg], psp_at_source_surface.lat[neg], color='blue', s=10,
label='Negative Polarity')
ax.scatter(psp_at_source_surface.lon[pos], psp_at_source_surface.lat[pos], color='red', s=10, 
label='Positive Polarity')

ax.set_title(str(pfss_br.date)[:-4] + r" $\rm R_{ss} =$" + str(rss))
ax.set_xlabel("Carrington Longitude (deg)")
ax.set_ylabel("Sine Carrington Latitude")

ax.set_xlim(0,360)
ax.legend(loc='lower right')

fig_title = str(pfss_br.date) + '_' + str(rss) + '.png'
figpath = os.path.join(IMG_DIR, fig_title)
fig.savefig(figpath, dpi=1200, bbox_inches='tight')

# Comparison Plotting

In [84]:
# do some comparison plotting:
# read in HCS CSV file
import pandas as pd
import matplotlib.pyplot as plt
hcs_file = '/Users/tamarervin/Desktop/Repositories/mag_model/data/gong/hcs_data.csv'
hcs = pd.read_csv(hcs_file, engine='python', index_col=0)

# read in PSP CSV file
psp_file = '/Users/tamarervin/Desktop/Repositories/mag_model/data/psp/psp_ss.csv'
psp = pd.read_csv(psp_file, engine='python', delimiter=',', index_col=0)


In [162]:
hcs_data = hcs_data[0:35]

In [1]:
# set up plotting grid
import numpy as np
fig, axs = plt.subplots(7, 5, sharey='row', sharex='col', figsize=[50, 35], gridspec_kw={'hspace': 0.05, 'wspace': 0.05})

for i, h in enumerate(hcs_data):
    head = h.columns[0]
    # for i, h in enumerate(hcs.columns):
    # determine the source surface radius and date
    rss, d = float(head.split('_')[0]), head.split('_')[1]
    lc = str(int(rss))

    # get indices for axs
    j = int((rss/100*4) - 8)
    k = int(i/7)

    # plot the HCS
    # hs = hcs.iloc[:, i]
    axs[j, k].scatter(list(h.iloc[0, :]), list(h.iloc[1, :]), color='black', s=3, label= r'$\rm R_{ss} = $' + str(float(rss/100)))

    # plot the PSP trajectory on top of it
    pos = np.where(psp_data.polarity == 1)[0]
    neg = np.where(psp_data.polarity == -1)[0]

    axs[j, k].scatter(coords.loc[:, 'lon_' + lc][neg], coords.loc[:, 'lat_' + lc][neg], color='blue', s=10)
    axs[j, k].scatter(coords.loc[:, 'lon_' + lc][pos], coords.loc[:, 'lat_' + lc][pos], color='red', s=10)

    # set y-label
    if k == 0:
        axs[j, k].set_ylabel("Sine Carrington Latitude")
    
    # set x label
    if j == 6:
        axs[j, k].set_xlabel("Carrington Longitude (deg)")

    # set title (date)
    if j == 0:
        axs[j, k].set_title(d.split('T')[0])

    # set x limit
    axs[j, k].set_xlim(0,360)
    axs[j, k].legend(loc='lower right')

fig_title = 'so_comparison.png'
figpath = os.path.join(IMG_DIR, fig_title)
fig.savefig(figpath, dpi=1200, bbox_inches='tight')
    

NameError: name 'plt' is not defined

In [ ]:
# get the radial data from stuart
# make the heliospheric current sheet on the 2D grid of Br -- contour where Br at SS is 0
# plot the PSP trajectory across the Br grid 
# use the timeseries from the data on the astrospice thing to get the actual 
#       PSP coordinates for comparison

In [148]:
# plt.scatter(list(h.iloc[0, :]), list(h.iloc[1, :]))
coords.loc[:, 'lon_' + lc][pos[0]]
# pos[0]

116     237.619033
373     210.165292
469     205.494928
470     205.481841
492     205.460587
           ...    
1283    329.859021
1284    329.773555
1286    329.602319
1287    329.516550
1298    328.566557
Name: lon_200, Length: 110, dtype: float64

# goals
- find good magnetogram
- change the Rss surface height

- want to create a csv file with HCS data, PSP data